# 04 - Geração de Respostas via LLM

Este notebook implementa a **Etapa 4** do pipeline SINKT: geração de respostas simuladas usando a API da OpenAI.

## Objetivo
Enriquecer as interações BKT com:
- Respostas textuais simuladas (para questões descritivas e múltipla escolha)
- Justificativas de erro baseadas no tipo de erro e resposta do aluno

## Entrada
- `data/output/notebooks/simulacao_interacoes/interactions_bkt.json`

## Saída
- `data/output/notebooks/geracao_respostas_llm/interactions_complete.json`

## Importação de Bibliotecas

In [25]:
import json
import os
import random
import time
from datetime import datetime
from typing import Dict, List, Any, Optional, Tuple
from dotenv import load_dotenv
from openai import OpenAI
import numpy as np

load_dotenv()

print("Bibliotecas importadas com sucesso")

Bibliotecas importadas com sucesso


## Configuração da API OpenAI

In [26]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY or OPENAI_API_KEY == "SUA_CHAVE_OPENAI_AQUI":
    raise ValueError("OPENAI_API_KEY nao configurada no arquivo .env")

client = OpenAI(api_key=OPENAI_API_KEY)

MODEL_NAME = "gpt-4.1-mini"
TEMPERATURE = 0.7
MAX_TOKENS = 300

print(f"OpenAI API configurada")
print(f"  - Modelo: {MODEL_NAME}")
print(f"  - Temperature: {TEMPERATURE}")
print(f"  - Max Tokens: {MAX_TOKENS}")

OpenAI API configurada
  - Modelo: gpt-4.1-mini
  - Temperature: 0.7
  - Max Tokens: 300


## Carregamento de Dados

In [27]:
INPUT_FILE = 'data/output/notebooks/simulacao_interacoes/interactions_bkt.json'
OUTPUT_DIR = 'data/output/notebooks/geracao_respostas_llm'
OUTPUT_FILE = os.path.join(OUTPUT_DIR, 'interactions_complete.json')
CHECKPOINT_FILE = os.path.join(OUTPUT_DIR, 'checkpoint.json')

os.makedirs(OUTPUT_DIR, exist_ok=True)

with open(INPUT_FILE, 'r', encoding='utf-8') as f:
    bkt_data = json.load(f)
interactions_bkt = bkt_data['interactions']

with open('data/json/questions_graph.json', 'r', encoding='utf-8') as f:
    questions_data = json.load(f)
questions_list = questions_data.get('questions', [])
questions_map = {q['id']: q for q in questions_list}

with open('data/json/concepts_graph.json', 'r', encoding='utf-8') as f:
    concepts_data = json.load(f)
concepts_list = concepts_data.get('concepts', [])
concepts_map = {c['id']: c for c in concepts_list}

with open('data/output/notebooks/geracao_estudantes/students.json', 'r', encoding='utf-8') as f:
    students_data = json.load(f)
students_list = students_data['students']
students_map = {s['id']: s for s in students_list}

print(f"Dados carregados:")
print(f"  - Interacoes BKT: {len(interactions_bkt)}")
print(f"  - Questoes: {len(questions_map)}")
print(f"  - Conceitos: {len(concepts_map)}")
print(f"  - Estudantes: {len(students_map)}")

Dados carregados:
  - Interacoes BKT: 4499
  - Questoes: 680
  - Conceitos: 251
  - Estudantes: 100


## Configuração de Parâmetros

In [28]:
CHECKPOINT_INTERVAL = 50
RATE_LIMIT_DELAY = 0.1
SEED = 42

ERROR_TYPE_DESCRIPTIONS = {
    'misconception': {
        'description': 'Confusao conceitual - estudante confundiu com conceito similar',
        'prompt_hint': 'Demonstre confusao entre conceitos similares, misturando definicoes ou aplicacoes'
    },
    'careless': {
        'description': 'Erro por descuido - estudante conhece mas nao prestou atencao',
        'prompt_hint': 'Responda de forma apressada, cometendo erros bobos que demonstram falta de atencao'
    },
    'slip': {
        'description': 'Erro por distracao - estudante sabe mas cometeu deslize',
        'prompt_hint': 'Mostre conhecimento parcial mas cometa um erro de digitacao ou troca de termos'
    },
    'incomplete': {
        'description': 'Resposta incompleta - faltam elementos importantes',
        'prompt_hint': 'Responda apenas parcialmente, omitindo partes importantes da explicacao'
    },
    'misunderstanding': {
        'description': 'Mal entendimento do enunciado',
        'prompt_hint': 'Responda como se tivesse entendido a pergunta de forma diferente'
    }
}

np.random.seed(SEED)
random.seed(SEED)

print(f"Configuracoes:")
print(f"  - Checkpoint a cada: {CHECKPOINT_INTERVAL} interacoes")
print(f"  - Rate limit delay: {RATE_LIMIT_DELAY}s")
print(f"  - Seed: {SEED}")

Configuracoes:
  - Checkpoint a cada: 50 interacoes
  - Rate limit delay: 0.1s
  - Seed: 42


## Funções de Geração de Respostas

In [29]:
def get_student_context(student_id: str) -> str:
    """Retorna contexto do estudante para o prompt."""
    student = students_map.get(student_id, {})
    profile_id = student.get('profile_id', 'balanced')
    
    profile_descriptions = {
        'quick_learner': 'aprende rapido, confiante, as vezes superficial',
        'careful': 'cauteloso, detalhista, prefere ter certeza',
        'struggling': 'tem dificuldades, inseguro, precisa de mais tempo',
        'intuitive': 'intuitivo, criativo, pode pular etapas',
        'logical': 'logico, metodico, segue procedimentos',
        'balanced': 'equilibrado, consistente, desempenho medio'
    }
    
    return profile_descriptions.get(profile_id, 'estudante padrao')


def generate_response_prompt(question: Dict, is_correct: bool, 
                            error_type: Optional[str], student_context: str) -> str:
    """Gera o prompt para a API baseado no contexto."""
    q_text = question.get('q', '')
    q_type = question.get('type', 'descriptive')
    concept_name = question.get('c_name', '')
    explanation = question.get('exp', '')
    
    base_prompt = f"""Voce esta simulando a resposta de um estudante de Linux/Shell Script.
Contexto do estudante: {student_context}
Conceito avaliado: {concept_name}
Pergunta: {q_text}

Informacao correta (para referencia): {explanation}
"""
    
    if q_type == 'multiple_choice':
        options = question.get('opt', {})
        correct_answer = question.get('ans', 'A')
        options_text = '\n'.join([f"{k}: {v}" for k, v in options.items()])
        
        if is_correct:
            return f"""{base_prompt}
Opcoes:
{options_text}

O estudante ACERTOU. Responda APENAS com "Opcao {correct_answer}" (nada mais)."""
        else:
            error_hint = ERROR_TYPE_DESCRIPTIONS.get(error_type, {}).get('prompt_hint', '')
            wrong_options = [k for k in options.keys() if k != correct_answer]
            wrong_choice = random.choice(wrong_options) if wrong_options else 'A'
            return f"""{base_prompt}
Opcoes:
{options_text}

O estudante ERROU (tipo: {error_type}). {error_hint}
Responda APENAS com "Opcao {wrong_choice}" (nada mais)."""
    
    else:  # descriptive
        if is_correct:
            return f"""{base_prompt}
O estudante ACERTOU a questao.
Gere uma resposta CORRETA e completa de 2-4 frases.
Escreva em linguagem natural de estudante, nao muito formal."""
        else:
            error_hint = ERROR_TYPE_DESCRIPTIONS.get(error_type, {}).get('prompt_hint', '')
            return f"""{base_prompt}
O estudante ERROU a questao (tipo de erro: {error_type}).
Instrucao especifica: {error_hint}
Gere uma resposta INCORRETA de 2-4 frases que demonstre esse tipo de erro.
Escreva em linguagem natural de estudante."""


def generate_error_explanation_prompt(question: Dict, error_type: str, 
                                      response: str, concept_name: str) -> str:
    """Gera prompt para justificativa do erro."""
    q_text = question.get('q', '')
    correct_info = question.get('exp', '')
    error_desc = ERROR_TYPE_DESCRIPTIONS.get(error_type, {}).get('description', 'Erro generico')
    
    return f"""Analise o erro do estudante e gere uma justificativa pedagogica.

Conceito: {concept_name}
Pergunta: {q_text}
Resposta do estudante: {response}
Tipo de erro: {error_type} ({error_desc})
Informacao correta: {correct_info}

Gere uma justificativa de 1-2 frases explicando:
1. O que o estudante errou especificamente
2. Qual conceito precisa ser reforçado

Seja objetivo e tecnico. Use terceira pessoa ("O estudante...")."""


print("Funcoes de geracao de prompt definidas")

Funcoes de geracao de prompt definidas


In [30]:
def call_openai_api(prompt: str, max_retries: int = 3) -> Optional[str]:
    """Chama a API da OpenAI com retry logic."""
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=MODEL_NAME,
                messages=[
                    {"role": "system", "content": "Voce simula respostas de estudantes de forma realista."},
                    {"role": "user", "content": prompt}
                ],
                temperature=TEMPERATURE,
                max_tokens=MAX_TOKENS
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            if attempt < max_retries - 1:
                wait_time = (attempt + 1) * 2
                print(f"  Erro na API (tentativa {attempt + 1}): {str(e)[:50]}. Aguardando {wait_time}s...")
                time.sleep(wait_time)
            else:
                print(f"  Erro apos {max_retries} tentativas: {str(e)[:100]}")
                return None
    return None


def generate_student_response(interaction: Dict, question: Dict) -> Tuple[str, Optional[str]]:
    """Gera resposta do estudante e justificativa de erro se aplicavel."""
    student_id = interaction['student_id']
    is_correct = interaction['is_correct']
    error_type = interaction.get('error_type')
    concept_name = question.get('c_name', '')
    
    student_context = get_student_context(student_id)
    
    # Gera resposta
    response_prompt = generate_response_prompt(question, is_correct, error_type, student_context)
    response = call_openai_api(response_prompt)
    
    if not response:
        response = "Nao sei responder essa pergunta." if not is_correct else "Resposta correta gerada."
    
    # Gera justificativa de erro se aplicavel
    error_explanation = None
    if not is_correct and error_type:
        explanation_prompt = generate_error_explanation_prompt(
            question, error_type, response, concept_name
        )
        error_explanation = call_openai_api(explanation_prompt)
        
        if not error_explanation:
            error_explanation = f"Estudante cometeu erro do tipo '{error_type}'. Necessario reforco em '{concept_name}'."
    
    return response, error_explanation


print("Funcoes de chamada da API definidas")

Funcoes de chamada da API definidas


## Sistema de Checkpoint

In [31]:
def save_checkpoint(processed_interactions: List[Dict], processed_count: int):
    """Salva checkpoint do progresso."""
    checkpoint_data = {
        'processed_count': processed_count,
        'timestamp': datetime.now().isoformat(),
        'interactions': processed_interactions
    }
    with open(CHECKPOINT_FILE, 'w', encoding='utf-8') as f:
        json.dump(checkpoint_data, f, indent=2, ensure_ascii=False)


def load_checkpoint() -> Tuple[List[Dict], int]:
    """Carrega checkpoint se existir."""
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, 'r', encoding='utf-8') as f:
            checkpoint_data = json.load(f)
        return checkpoint_data['interactions'], checkpoint_data['processed_count']
    return [], 0


def remove_checkpoint():
    """Remove arquivo de checkpoint."""
    if os.path.exists(CHECKPOINT_FILE):
        os.remove(CHECKPOINT_FILE)
        print("Checkpoint removido (processamento completo)")


print("Sistema de checkpoint configurado")

Sistema de checkpoint configurado


## Processamento das Interações

In [32]:
processed_interactions, start_idx = load_checkpoint()

if start_idx > 0:
    print(f"Retomando do checkpoint: {start_idx}/{len(interactions_bkt)} interacoes processadas")
else:
    print(f"Iniciando processamento de {len(interactions_bkt)} interacoes...")

start_time = time.time()
error_count = 0
success_count = 0

for idx in range(start_idx, len(interactions_bkt)):
    interaction = interactions_bkt[idx]
    question_id = interaction['question_id']
    question = questions_map.get(question_id)
    
    if not question:
        print(f"  Questao nao encontrada: {question_id}")
        continue
    
    # Gera resposta via LLM
    response, error_explanation = generate_student_response(interaction, question)
    
    # Calcula tempo simulado (baseado em dificuldade e tipo)
    base_time = 60 if question.get('type') == 'multiple_choice' else 180
    difficulty_mult = {'easy': 0.7, 'medium': 1.0, 'hard': 1.4}.get(question.get('diff', 'medium'), 1.0)
    time_spent = int(base_time * difficulty_mult * (0.8 + random.random() * 0.4))
    
    # Monta interacao completa
    complete_interaction = {
        'interaction_id': f"int_{idx:06d}",
        'student_id': interaction['student_id'],
        'question_id': question_id,
        'concept_id': question.get('c_id', ''),
        'question_type': question.get('type', 'descriptive'),
        'timestamp': interaction['timestamp'],
        'response': response,
        'is_correct': interaction['is_correct'],
        'error_type': interaction.get('error_type'),
        'error_explanation': error_explanation,
        'mastery_before': interaction['mastery_before'],
        'mastery_after': interaction['mastery_after'],
        'time_spent_seconds': time_spent
    }
    
    processed_interactions.append(complete_interaction)
    success_count += 1
    
    # Rate limiting
    time.sleep(RATE_LIMIT_DELAY)
    
    # Checkpoint e log de progresso
    current_count = idx + 1
    if current_count % CHECKPOINT_INTERVAL == 0:
        save_checkpoint(processed_interactions, current_count)
        elapsed = time.time() - start_time
        rate = current_count / elapsed if elapsed > 0 else 0
        eta = (len(interactions_bkt) - current_count) / rate if rate > 0 else 0
        print(f"Checkpoint: {current_count}/{len(interactions_bkt)} ({current_count/len(interactions_bkt)*100:.1f}%) - ETA: {eta/60:.1f}min")

elapsed_time = time.time() - start_time
print(f"\nProcessamento concluido!")
print(f"  - Tempo total: {elapsed_time/60:.1f} min")
print(f"  - Interacoes processadas: {len(processed_interactions)}")
print(f"  - Taxa: {len(processed_interactions)/elapsed_time:.2f} int/s")

Iniciando processamento de 4499 interacoes...
Checkpoint: 50/4499 (1.1%) - ETA: 192.8min
Checkpoint: 100/4499 (2.2%) - ETA: 165.3min
Checkpoint: 150/4499 (3.3%) - ETA: 163.8min
Checkpoint: 200/4499 (4.4%) - ETA: 151.8min
Checkpoint: 250/4499 (5.6%) - ETA: 147.1min
Checkpoint: 300/4499 (6.7%) - ETA: 145.9min
Checkpoint: 350/4499 (7.8%) - ETA: 145.9min
Checkpoint: 400/4499 (8.9%) - ETA: 137.6min
Checkpoint: 450/4499 (10.0%) - ETA: 138.3min
Checkpoint: 500/4499 (11.1%) - ETA: 138.7min
Checkpoint: 550/4499 (12.2%) - ETA: 142.1min
Checkpoint: 600/4499 (13.3%) - ETA: 139.9min
Checkpoint: 650/4499 (14.4%) - ETA: 141.2min
Checkpoint: 700/4499 (15.6%) - ETA: 140.3min
Checkpoint: 750/4499 (16.7%) - ETA: 137.0min
Checkpoint: 800/4499 (17.8%) - ETA: 133.9min
Checkpoint: 850/4499 (18.9%) - ETA: 131.5min
Checkpoint: 900/4499 (20.0%) - ETA: 128.4min
Checkpoint: 950/4499 (21.1%) - ETA: 126.2min
Checkpoint: 1000/4499 (22.2%) - ETA: 124.7min
Checkpoint: 1050/4499 (23.3%) - ETA: 122.7min
Checkpoint: 1100

## Salvamento dos Resultados

In [33]:
# Calcula metricas de qualidade
total_interactions = len(processed_interactions)
correct_count = sum(1 for i in processed_interactions if i['is_correct'])
accuracy = correct_count / total_interactions if total_interactions > 0 else 0

error_distribution = {}
for interaction in processed_interactions:
    error_type = interaction.get('error_type')
    if error_type:
        error_distribution[error_type] = error_distribution.get(error_type, 0) + 1

student_counts = {}
for interaction in processed_interactions:
    sid = interaction['student_id']
    student_counts[sid] = student_counts.get(sid, 0) + 1

valid_responses = sum(1 for i in processed_interactions if i.get('response') and len(i['response']) > 5)

# Monta output final
output_data = {
    "metadata": {
        "description": "Conjunto de interacoes simuladas com respostas geradas por LLM para estudantes SINKT",
        "version": "2.0.0",
        "created_at": datetime.now().isoformat(),
        "llm_model": MODEL_NAME,
        "llm_temperature": TEMPERATURE,
        "total_interactions": total_interactions,
        "total_students": len(student_counts),
        "avg_interactions_per_student": total_interactions / len(student_counts) if student_counts else 0,
        "accuracy": accuracy,
        "error_types": list(ERROR_TYPE_DESCRIPTIONS.keys()),
        "quality_metrics": {
            "total_interactions": total_interactions,
            "total_students": len(student_counts),
            "avg_interactions_per_student": total_interactions / len(student_counts) if student_counts else 0,
            "correct_interactions": correct_count,
            "accuracy": accuracy,
            "error_distribution": error_distribution,
            "interactions_per_student_stats": {
                "min": min(student_counts.values()) if student_counts else 0,
                "max": max(student_counts.values()) if student_counts else 0,
                "mean": total_interactions / len(student_counts) if student_counts else 0
            },
            "response_quality": {
                "total_responses": total_interactions,
                "empty_responses": total_interactions - valid_responses,
                "valid_responses": valid_responses,
                "validity_percentage": (valid_responses / total_interactions * 100) if total_interactions else 0
            }
        }
    },
    "interactions": processed_interactions
}

with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

remove_checkpoint()

print(f"\nResultados salvos em: {OUTPUT_FILE}")
print(f"Tamanho do arquivo: {os.path.getsize(OUTPUT_FILE) / (1024*1024):.2f} MB")

Checkpoint removido (processamento completo)

Resultados salvos em: data/output/notebooks/geracao_respostas_llm/interactions_complete.json
Tamanho do arquivo: 3.27 MB


## Análise de Resultados

In [34]:
print("\n" + "="*70)
print("GERACAO DE RESPOSTAS LLM CONCLUIDA")
print("="*70)

print(f"\nEstatisticas Gerais:")
print(f"  - Total de interacoes: {total_interactions}")
print(f"  - Estudantes: {len(student_counts)}")
print(f"  - Acuracia: {accuracy:.1%}")
print(f"  - Respostas validas: {valid_responses} ({valid_responses/total_interactions*100:.1f}%)")

print(f"\nDistribuicao de Erros:")
for error_type, count in sorted(error_distribution.items()):
    pct = count / (total_interactions - correct_count) * 100 if (total_interactions - correct_count) > 0 else 0
    print(f"  - {error_type}: {count} ({pct:.1f}%)")

print("\n" + "="*70)


GERACAO DE RESPOSTAS LLM CONCLUIDA

Estatisticas Gerais:
  - Total de interacoes: 4499
  - Estudantes: 100
  - Acuracia: 41.7%
  - Respostas validas: 4499 (100.0%)

Distribuicao de Erros:
  - careless: 520 (19.8%)
  - incomplete: 533 (20.3%)
  - misconception: 496 (18.9%)
  - misunderstanding: 512 (19.5%)
  - slip: 561 (21.4%)



## Exemplos de Interações Geradas

In [35]:
print("\nExemplos de Interacoes Geradas:")
print("-" * 70)

# Exemplo de acerto
correct_examples = [i for i in processed_interactions if i['is_correct']][:2]
for ex in correct_examples:
    print(f"\n[ACERTO] {ex['student_id']} - {ex['question_id']}")
    print(f"  Tipo: {ex['question_type']}")
    print(f"  Resposta: {ex['response'][:150]}..." if len(ex['response']) > 150 else f"  Resposta: {ex['response']}")

print("\n" + "-" * 70)

# Exemplos de erro (um de cada tipo)
shown_types = set()
for ex in processed_interactions:
    if not ex['is_correct'] and ex.get('error_type') and ex['error_type'] not in shown_types:
        print(f"\n[ERRO: {ex['error_type']}] {ex['student_id']} - {ex['question_id']}")
        print(f"  Tipo: {ex['question_type']}")
        print(f"  Resposta: {ex['response'][:150]}..." if len(ex['response']) > 150 else f"  Resposta: {ex['response']}")
        print(f"  Justificativa: {ex.get('error_explanation', 'N/A')}")
        shown_types.add(ex['error_type'])
        if len(shown_types) >= 3:
            break

print("\n" + "="*70)


Exemplos de Interacoes Geradas:
----------------------------------------------------------------------

[ACERTO] student_0000 - concept_001_q3
  Tipo: multiple_choice
  Resposta: Opcao A

[ACERTO] student_0000 - concept_088_q4
  Tipo: descriptive
  Resposta: Deepin é uma distribuição Linux que se destaca pela sua interface bonita e fácil de usar, o que ajuda muito quem está começando ou quer algo mais intu...

----------------------------------------------------------------------

[ERRO: slip] student_0000 - concept_055_q4
  Tipo: descriptive
  Resposta: O /tmp é um diretório usado para armazenar arquivos temporários criados pelos programas durante a execução. Ele é importante porque ajuda a guardar da...
  Justificativa: O estudante errou ao afirmar que o uso do /tmp evita que a memória do sistema fique cheia, confundindo armazenamento temporário em disco com gerenciamento de memória RAM. É necessário reforçar o conceito de que /tmp é um diretório em disco para arquivos temporários, 